### 発言小町の恋愛・結婚・離婚トピックから新着順に1000件取得する
#### 2020/01/01時点でのページの状況を元に作成しているためページの構造が変わると機能しないので注意

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_contents(tar_link):
    # 対象の投稿のURLを渡すと表の作成に必要な情報を返す関数
    post_res = requests.get(tar_link)
    post_soup = BeautifulSoup(post_res.content, 'html.parser')
    
    title = post_soup.find("div", class_="main-contents-inner-topi-head-title").find("h1").text
    content = post_soup.find("div", class_="main-contents-inner-topi-article-text").find("p").text
    post_time = post_soup.find("div", class_="mass").text
    res_num = int(post_soup.find("div", class_="res").find("span").text.replace("," , "")) # 3桁区切りの数字に対応するため","を置換
    fav_num = int(post_soup.find("div", class_="fav").find("span").text.replace("," , "")) # 3桁区切りの数字に対応するため","を置換
    
    return title, content, post_time, res_num, fav_num

In [3]:
base_url = "https://komachi.yomiuri.co.jp/" # 発言小町のURL
# 新着一覧-更新時間順のページのURL
# g=02;キャリア, g=04:恋愛・結婚・離婚
tar_top_url = base_url + "topics/?g=02&p=" # スクレイピング対象のトップページ

# 取得したい項目のリストを初期化
title_list = []
content_list = []
post_time_list = []
res_num_list = []
fav_num_list = []

for p in range(1, 21):  # 1ページあたり50, 1000件が対象のため20ページ存在する前提
    # スクレイピング対象の URL にリクエストを送り HTML を取得する
    res = requests.get(tar_top_url + str(p))

    # レスポンスの HTML から BeautifulSoup オブジェクトを作る
    soup = BeautifulSoup(res.content, 'html.parser')

    # リンクを抽出したいクラスにオブジェクトの範囲を絞る
    tar_soup = soup.find("div", class_="main-contents-inner-new-list switching_contents js-active")

    # 上位50件のみリンクを取得する # ToDo:もっと良い書き方はあるはず
    links = [url.get('href') for url in tar_soup.find_all('a')][:50]

    for i in range(len(links)):
        tar_link = base_url + links[i]
        title, content, post_time, res_num, fav_num = get_contents(tar_link)
        title_list.append(title)
        content_list.append(content)
        post_time_list .append(post_time)
        res_num_list.append(res_num)
        fav_num_list .append(fav_num)
    

In [4]:
out_df = pd.DataFrame({"title":title_list, "content":content_list, "post_time":post_time_list, "res":res_num_list, "fav":fav_num_list})
out_df[["title", "content", "post_time", "res", "fav"]]

,title,content,post_time,res,fav
0,夫の今後。定年前の早期退職か？そのまま再雇用か？,50代のご主人をお持ちの方、またはご本人様にお伺いします。\n夫は53歳ですが、基本定年は6...,2019年11月22日 3:10,25,71
1,共働きしたくありません,24歳、事務員をしております。現在婚約している身で、来年結婚することになりました。\n結婚自...,2019年12月26日 18:01,62,171
2,新しい職場が不安です。,1月から新しい職場になりますが不安でいっぱいです。どうか励ましてほしいです。\n先週、派遣会...,2019年12月31日 7:26,3,1
3,孫の世話で忙しい。,50代、3歳間近の孫と同居しているおばあちゃんです。\nパートで事務をやっていまして\n週5...,2019年12月27日 19:38,69,415
4,退職すべきか否かのご相談（社長に直訴してしまいました）,短大を卒業し、地元企業で新卒4年目です。\n私的に耐え難いと思い、退職想定で社長へ直訴しまし...,2019年12月29日 17:06,7,75
5,派遣の営業担当との月1面談、断われますか?,派遣として働きはじめて４ヶ月めです。\n派遣先の職場に、月1の頻度で派遣元の営業担当が面談に...,2019年12月22日 15:28,14,22
6,雇い止めをちらつかせる圧迫面接,閲覧ありがとうございます。62歳有期雇用の女性です。\n先日直属の上司とさらに上の部署の役職...,2019年12月18日 21:59,21,70
7,泥棒のような人,初めまして。\n職場での事、相談させて下さい。\n私の下に派遣の女性Aさん(50代)が、12...,2019年12月29日 8:02,19,211
8,公立の臨採か、私立の非常勤か,現在臨時的任用職員として公立学校で働く養護教諭一年目の者です。※今年度で任期満了\n\nつい...,2019年12月24日 18:42,7,12
9,小さな町での風評被害,小さな町に1年ほど前に越してきて、忙しい定食屋で働いています。\n\nこの仕事場に、Aと私の...,2019年12月31日 11:37,0,2


In [5]:
out_df.shape

(1000, 5)

In [6]:
out_df.to_csv("./out/komachi_work_20200101.csv", index=False)